# APPLICATION OF FACEBOOK' S P ROPHET ALGORITHM FOR SUCCESSFUL SALES FORECASTING BASED ON REAL - WORLD DATA

## Requirements

In [18]:
#pip install pandas
#pip install fbprophet
#pip install plotly

## Importing required modules

In [19]:
# Import modules
import pandas as pd 
from fbprophet import Prophet 
from fbprophet.plot import add_changepoints_to_plot
from sklearn.metrics import mean_squared_error, mean_absolute_error 
import numpy as np 


## Reading the CSV - Avocado Sales

In [20]:
# Read Data
data = pd.read_csv("data/avocado.csv")
data.head()

AttributeError: 'NoneType' object has no attribute 'items'

   Unnamed: 0        Date  AveragePrice  Total Volume     4046       4225  \
0           0  12/27/2015          1.33      64236.62  1036.74   54454.85   
1           1  12/20/2015          1.35      54876.98   674.28   44638.81   
2           2  12/13/2015          0.93     118220.22   794.70  109149.67   
3           3   12/6/2015          1.08      78992.15  1132.00   71976.41   
4           4  11/29/2015          1.28      51039.60   941.48   43838.39   

     4770  Total Bags  Small Bags  Large Bags  XLarge Bags          type  \
0   48.16     8696.87     8603.62       93.25          0.0  conventional   
1   58.33     9505.56     9408.07       97.49          0.0  conventional   
2  130.50     8145.35     8042.21      103.14          0.0  conventional   
3   72.58     5811.16     5677.40      133.76          0.0  conventional   
4   75.78     6183.95     5986.26      197.69          0.0  conventional   

   year  region  
0  2015  Albany  
1  2015  Albany  
2  2015  Albany  
3  2015 

### Data Preprocessing

We should aggregate data accordingly to our objective. In this example we will aggregate data per month since it is a good scale to analyse the sails of a fruit.

### Feature Selection

Our dataset has information relating several avocado types as well as volume of sails and price. 

On the recomended paper it is said: 

"

Based on industry experience and discussion with clients in real field, it was concluded that
several different criteria can be used to sort the product portfolio by relevance:

1. the total profit per product over the last year (if the profit per sale is available, which is
a rarity),
2. the total financial turnover (i.e. net sales) per product over the last year (if the unit price
per sale is available, which is often a case),
3. the total quantity sold per item over the last year (if none of the aforementioned data is
available).

"

Since our dataset does not have any information relating the profit per sail, we will have to go for the second option which is to use the total financial turnover.


In [14]:
data['Date'] = data['Date'].str[:-3] #Remove day
data['TurnOver'] = data['AveragePrice'] * data['Total Volume']   #Turn over

aggregation_functions = {'TurnOver': 'sum'}
data = data.groupby(data['Date']).aggregate(aggregation_functions).reset_index()
data.head()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## Creating the dataframe
We convert the data into the convention of ds (date timeseries) and y (output) to be predicted.  
In this case, it is the number of monthly car sales.

In [ ]:
df = pd.DataFrame() 
df['ds'] = pd.to_datetime(data['Date']) 
df['y'] = data['TurnOver'] 
df.head()

To test our model we decided to split the data in train and test where the test data will be the last 3 months

In [ ]:
split_date = "2018-01-01"
df_train = df[df['ds'] < split_date].copy()
df_test = df[df['ds'] >= split_date].copy()


## The model - GAM
The model used in prohpet is called General Additive model - (GAM), and it is a generalized linear model in which the linear response variable depends linearly on unknown smooth functions of some predictor variables, and interest focuses on inference about these smooth functions, so it basically models a time-series as the sum of different components (non-linear trend, periodic components and holidays or special events) and allows to incorporate extra-regressors (categorical or continuous).

## Initialize the model
The first step in the Facebook Prophet is to instantiate the model, it is here that you can set the prior scales for each component of your time-series, as well as the number of Fourier series to use to model the sales components. In this case we dont add any extras such as the country holidays or seasonality mode, since the datasets country wasnt provided and therefore we cant make assumptions about the country.

To pick the scale, the general rule is that larger prior scales and larger number of Fourier series will make the model more flexible, but at the potential cost of generalisation: i.e. the model might overfit, learning the noise (rather than the signal) in the training data, but giving poor results when applied to yet unseen data (the test data).  
Setting these hyperparameters is a fine tuning that is specific to the problem, and is sometimes the most time consuming task that clearly improves the results.

In [ ]:
m = Prophet(weekly_seasonality=True, \
            daily_seasonality=False) 
m.fit(df_train) 

## Testing

After the model fitted the training dataset we compared the results with the real values

In [ ]:
forecast = m.predict(df_test) 
forecast.tail()

MAPE is used to quantify the overall accuracy of the forecasting framework and calculate the expected
level of reliability.  
According to the paper approximately 50% of the products (with a sufficiently long historical data) can be forecasted with MAPE < 30% on monthly basis.   
Also, the the best selling products will have a lower MAPE.

In [ ]:
def mean_abs_perc_err(y_true, y_pred):  
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Mean Squared Error (MSE):", mean_squared_error(y_true = df_test["y"], y_pred = forecast['yhat'])) 
print("Mean Absolute Error (MAE):", mean_absolute_error(y_true = df_test["y"], y_pred = forecast['yhat'])) 
print("Mean Absolute % Error (MAPE): ", mean_abs_perc_err(y_true = np.asarray(df_test["y"]), y_pred = np.asarray(forecast['yhat'])))

## Generate the future dataframe
Here, we create predictions for the next 5 years, and since the frequency is monthly, we need to create 12*5 periods

In [ ]:
m = Prophet(weekly_seasonality=True, \
            daily_seasonality=False) 
m.fit(df) #retrain for all the dataset 

future = m.make_future_dataframe(periods=12*5, freq='M')

## Analyze predictions

### Prophet forecasts

In [ ]:
forecast = m.predict(future) 
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']].tail()

### Plotting the model

The below image shows the basic prediction. The light blue is the uncertainty level(yhat_upper and yhat_lower), the dark blue is the prediction(yhat) and the black dots are the original data. We can see that the predicted data is very close to the actual data. In the last five years, there is no “actual” data, but looking at the performance of our model in years where data is available it is safe to say that the predictions are close to accurate.

In [ ]:
fig2 = m.plot_components(forecast) 

### Adding changepoints
The changepoints could be specified by the analyst using known dates of product launches
and other growth-altering events, or may be automatically selected given a set of candidates.
Automatic selection can be done quite naturally as we demonstrate below

In [ ]:
# Adding changepoints

fig = m.plot(forecast) 
a = add_changepoints_to_plot(fig.gca(), m, forecast)

## Using the model in a different way

Prophet can be used in a variety of ways to best predict what a certain company decides to predict about their sales. In the last example we perdicted the monthly financial turnover but,  if required, this tool can be used to predict other parameters.  

As another example we will predict the daily volume of products sold for the next week, based on the last month of sales.

Since we are predicting the total number of sales daily, the model will be better if we train it with daily sales as well.

In [ ]:
# Read Data
data = pd.read_csv("data/avocado.csv")   #Data was aggregated, need to read it again
data = data.tail(30)#last 30 days

df = pd.DataFrame() 
df['ds'] = pd.to_datetime(data['Date']) 
df['y'] = data['Total Volume'] 

m = Prophet() 
m.fit(df)
future = m.make_future_dataframe(periods=7, freq='D')  #Predict the next 20 days
forecast = m.predict(future) 
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']].tail()

fig = m.plot(forecast) 
fig2 = m.plot_components(forecast) 
fig3 = m.plot(forecast) 
a = add_changepoints_to_plot(fig.gca(), m, forecast)
